In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
csv_files = glob.glob('*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

['HUGO LOPEZ-GATELL RAMIREZ1.csv', 'HUGO LOPEZ-GATELL RAMIREZ10.csv', 'HUGO LOPEZ-GATELL RAMIREZ11.csv', 'HUGO LOPEZ-GATELL RAMIREZ12.csv', 'HUGO LOPEZ-GATELL RAMIREZ13.csv', 'HUGO LOPEZ-GATELL RAMIREZ14.csv', 'HUGO LOPEZ-GATELL RAMIREZ15.csv', 'HUGO LOPEZ-GATELL RAMIREZ16.csv', 'HUGO LOPEZ-GATELL RAMIREZ17.csv', 'HUGO LOPEZ-GATELL RAMIREZ18.csv', 'HUGO LOPEZ-GATELL RAMIREZ19.csv', 'HUGO LOPEZ-GATELL RAMIREZ2.csv', 'HUGO LOPEZ-GATELL RAMIREZ20.csv', 'HUGO LOPEZ-GATELL RAMIREZ21.csv', 'HUGO LOPEZ-GATELL RAMIREZ22.csv', 'HUGO LOPEZ-GATELL RAMIREZ3.csv', 'HUGO LOPEZ-GATELL RAMIREZ4.csv', 'HUGO LOPEZ-GATELL RAMIREZ5.csv', 'HUGO LOPEZ-GATELL RAMIREZ6.csv', 'HUGO LOPEZ-GATELL RAMIREZ7.csv', 'HUGO LOPEZ-GATELL RAMIREZ8.csv', 'HUGO LOPEZ-GATELL RAMIREZ9.csv']


In [3]:
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y 
# el resultado final será la lista dataframes
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola
list_data
 
pd.concat(list_data,ignore_index=True)['Texto']

0                     Muchas gracias, doctor Reyes Terán.
1       Ahora le pido al doctor José Luis Alomía si no...
2                                 Gracias, doctor Alomía.
3       Abrimos para preguntas. Por favor, compañera, ...
4        Le contesto la segunda y le paso a su compatr...
                              ...                        
2049    No hay una previsión anticipada de cierre de a...
2050    Y aquí ratifico la instrucción del señor presi...
2051                                             Gracias.
2052                                                  ---
2053                                              VE 823.
Name: Texto, Length: 2054, dtype: object

In [4]:
informes = pd.concat(list_data,ignore_index=True)['Texto']
informes

0                     Muchas gracias, doctor Reyes Terán.
1       Ahora le pido al doctor José Luis Alomía si no...
2                                 Gracias, doctor Alomía.
3       Abrimos para preguntas. Por favor, compañera, ...
4        Le contesto la segunda y le paso a su compatr...
                              ...                        
2049    No hay una previsión anticipada de cierre de a...
2050    Y aquí ratifico la instrucción del señor presi...
2051                                             Gracias.
2052                                                  ---
2053                                              VE 823.
Name: Texto, Length: 2054, dtype: object

In [5]:
type(informes)

pandas.core.series.Series

In [6]:
informe = np.array(informes)

In [7]:
informe

array([' Muchas gracias, doctor Reyes Terán.',
       'Ahora le pido al doctor José Luis Alomía si nos resume la situación actual en México.',
       ' Gracias, doctor Alomía.', ..., 'Gracias.', '---', 'VE 823.'],
      dtype=object)

In [8]:
import random

informe[random.randint(0, 2053)]

'Como se puede mostrar en múltiples evidencias, incluida la vivencial de todas estas conferencias de prensa, aquí está la información, está el micrositio, están las comunicaciones que tenemos en los estados, están las minutas de los comités, subcomités y ahora el consejo.'

#librea de tokenizacion

import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

informe2 = informe[random.randint(0, 2053)]

# instantiate tokenizer class
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

# tokenize tweets
informe_tokens = tokenizer.tokenize(informe2)

print()
print('Tokenized string:')
print(informe2)

#Import the english stop words list from NLTK
stopwords_spanish = stopwords.words('spanish') 

print('Stop words\n')
print(stopwords_spanish)

print('\nPunctuation\n')
print(string.punctuation)

print()
print('\033[92m')
print(informe_tokens)
print('\033[94m')

informe_clean = []

for word in informe_tokens: # Go through every word in your tokens list
    if (word not in stopwords_spanish and  # remove stopwords
        word not in string.punctuation):  # remove punctuation
        informe_clean.append(word)

print('removed stop words and punctuation:')
print(informe_clean)

In [9]:
# libreta de lda

import spacy
nlp = spacy.load("es_core_news_sm")

In [10]:
def procesa_informe(doc): 
    
    return [
        token.norm_ for token in doc
        if token.is_alpha and not token.like_num and not token.is_stop and
           token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']
    ]

informe_tratado = [procesa_informe(doc) for doc in nlp.pipe(informes)]

In [11]:
i = random.randint(0, 2053)
print(i)
print(informes[i])
print("********************")
print(informe_tratado[i])

984
Entonces, esta medida no es exclusiva para personas en alto riesgo de complicarse, es para todas y todos, cualquier edad, cualquier condición de salud y desde luego embarazadas y no embarazadas. Entonces, las dos medidas se complementan perfectivamente.
********************
['medida', 'exclusiva', 'personas', 'alto', 'riesgo', 'complicarse', 'edad', 'condición', 'salud', 'embarazadas', 'embarazadas', 'medidas', 'complementan']


In [12]:
import warnings
warnings.filterwarnings('ignore')

from gensim import models, corpora

# Genera el diccionario de palabras
diccionario = corpora.Dictionary(informe_tratado)
diccionario.filter_extremes(no_below=5, no_above=0.3)
 
# Extrae las características en forma de BOW
corpus = [diccionario.doc2bow(doc) for doc in informe_tratado]

In [13]:
# Número de tópicos
n_topicos = 15

# Genera el modelo LDA
modelo_lda = models.LdaModel(corpus=corpus, num_topics=n_topicos, id2word=diccionario, iterations=100, passes=10)

# Si quieres guardar el modelo se hace esto
#modelo_lda.save("sonetosLDA.model")

In [14]:
modelo_hdp = models.hdpmodel.HdpModel(corpus, diccionario, T=n_topicos)

In [15]:
for (i, topico) in modelo_lda.print_topics(num_topics=n_topicos, num_words=6):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.043*"sitio" + 0.043*"productos" + 0.039*"turismo" + 0.036*"méxico" + 0.032*"doctor" + 0.029*"italia"
----------topico 1--------------------
0.091*"países" + 0.058*"méxico" + 0.036*"situación" + 0.033*"interés" + 0.032*"relación" + 0.031*"epidemias"
----------topico 2--------------------
0.052*"unidos" + 0.045*"insumos" + 0.045*"reserva" + 0.043*"llegada" + 0.032*"minutos" + 0.032*"mecanismos"
----------topico 3--------------------
0.094*"salud" + 0.055*"riesgo" + 0.042*"años" + 0.032*"pública" + 0.029*"ocurre" + 0.029*"seguridad"
----------topico 4--------------------
0.060*"proceso" + 0.048*"pendiente" + 0.034*"secretaría" + 0.033*"ve" + 0.032*"secretario" + 0.032*"masiva"
----------topico 5--------------------
0.083*"méxico" + 0.044*"país" + 0.038*"internacional" + 0.036*"viajes" + 0.025*"pregunta" + 0.024*"comunicación"
----------topico 6--------------------
0.060*"caso" + 0.056*"país" + 0.045*"dijimos" + 0.041*"llega" + 0.032*"virus" + 0.029

In [16]:
for (i, topico) in modelo_hdp.print_topics(num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.007*operaciones + 0.006*común + 0.005*cortés + 0.005*clases + 0.005*infecciones
----------topico 1--------------------
0.006*considerar + 0.005*punto + 0.005*fenómenos + 0.004*cosa + 0.004*epidemiología
----------topico 2--------------------
0.005*salir + 0.005*surge + 0.005*distintas + 0.005*vacaciones + 0.004*bioseguridad
----------topico 3--------------------
0.008*enorme + 0.006*niveles + 0.005*pone + 0.005*completo + 0.005*explico
----------topico 4--------------------
0.007*aprovechar + 0.005*retos + 0.005*eventos + 0.004*declarar + 0.004*comunidad
----------topico 5--------------------
0.006*esperar + 0.005*respiratorias + 0.005*componente + 0.005*bienestar + 0.005*actividad
----------topico 6--------------------
0.007*especialista + 0.005*llegara + 0.005*vigor + 0.004*pulso + 0.004*reglas
----------topico 7--------------------
0.008*gobiernos + 0.005*gobernador + 0.005*identifican + 0.005*acabo + 0.005*economías
----------topico 8-------

In [17]:
i = random.randint(0, 2053)

respuesta = informes[i]
ids = diccionario.doc2bow(informe_tratado[i])

print("El informe: \n\n" + respuesta)

topicos_lda = modelo_lda[ids]
topicos_lda.sort(key=lambda x:x[1], reverse=True)

topicos_hdp = modelo_hdp[ids]
topicos_hdp.sort(key=lambda x:x[1], reverse=True)

print("Pertenece a los tópicos (con el modelo LDA)")
for (topico, peso) in topicos_lda:
    print("\tTopico {} con un peso de {}".format(topico, peso))

print("Pertenece a los tópicos (con el modelo HDP)")
for (topico, peso) in topicos_hdp:
    print("\tTopico {} con un peso de {}".format(topico, peso))

El informe: 

 Gracias, doctor Gabastou.
Pertenece a los tópicos (con el modelo LDA)
	Topico 8 con un peso de 0.7666659355163574
	Topico 5 con un peso de 0.01666673831641674
	Topico 0 con un peso de 0.016666723415255547
	Topico 3 con un peso de 0.016666721552610397
	Topico 6 con un peso de 0.016666721552610397
	Topico 1 con un peso de 0.0166667178273201
	Topico 2 con un peso de 0.0166667178273201
	Topico 4 con un peso de 0.0166667178273201
	Topico 7 con un peso de 0.0166667178273201
	Topico 9 con un peso de 0.0166667178273201
	Topico 10 con un peso de 0.0166667178273201
	Topico 11 con un peso de 0.0166667178273201
	Topico 12 con un peso de 0.0166667178273201
	Topico 13 con un peso de 0.0166667178273201
	Topico 14 con un peso de 0.0166667178273201
Pertenece a los tópicos (con el modelo HDP)
	Topico 12 con un peso de 0.7678620605269209
	Topico 4 con un peso de 0.018977263925399824
	Topico 11 con un peso de 0.017599807906071536
	Topico 1 con un peso de 0.017513425784989274
	Topico 13 con 

In [18]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(modelo_lda, corpus, diccionario)
pyLDAvis.display(vis_data)

In [26]:
vis_data = gensimvis.prepare(modelo_hdp, corpus, diccionario)
pyLDAvis.display(vis_data)

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValidationError: 
 * Not all rows (distributions) in doc_topic_dists sum to 1.

In [20]:
print(f"El logaritomo de la perplejidad del corpus al modelo LDA es {modelo_lda.log_perplexity(corpus)}")

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


El logaritomo de la perplejidad del corpus al modelo LDA es -7.240753170744475


In [21]:
cm_lda = models.CoherenceModel(model=modelo_lda, texts=informe_tratado, coherence='c_v')
cm_hdp = models.CoherenceModel(model=modelo_hdp, texts=informe_tratado, coherence='c_v')


print("Coherencia del modelo LDA (C_V): {}".format(cm_lda.get_coherence()))
for (topico, coherencia) in enumerate(cm_lda.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

print("\n\nCoherencia del modelo HDP (C_V): {}".format(cm_hdp.get_coherence()))
for (topico, coherencia) in enumerate(cm_hdp.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherencia del modelo LDA (C_V): 0.40130602980591024
	Tópico 0, coherencia 0.47974602222532986
	Tópico 1, coherencia 0.3319774470114377
	Tópico 2, coherencia 0.435758763163874
	Tópico 3, coherencia 0.2707477562277444
	Tópico 4, coherencia 0.395754835870843
	Tópico 5, coherencia 0.5626785032379846
	Tópico 6, coherencia 0.38894851900817506
	Tópico 7, coherencia 0.43823495712136734
	Tópico 8, coherencia 0.6141576368047698
	Tópico 9, coherencia 0.34264613233577035
	Tópico 10, coherencia 0.23343477253535871
	Tópico 11, coherencia 0.36730149694195957
	Tópico 12, coherencia 0.37287505180206215
	Tópico 13, coherencia 0.38700938023390313
	Tópico 14, coherencia 0.39831917256807287


Coherencia del modelo HDP (C_V): 0.7530512356836855
	Tópico 0, coherencia 0.7333726267630534
	Tópico 1, coherencia 0.7350453430807942
	Tópico 2, coherencia 0.7774004878147425
	Tópico 3, coherencia 0.7874313066140313
	Tópico 4, coherencia 0.7415655431308665
	Tópico 5, coherencia 0.7198090048767981
	Tópico 6, coherenci

In [22]:
print("Las palabra que definen los tópicos en LDA\n")
t_palabras = cm_lda.top_topics_as_word_lists(model=modelo_lda, dictionary=diccionario)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")
    
print("\n\nLas palabra que definen los tópicos en HDP\n")
t_palabras = cm_hdp.top_topics_as_word_lists(model=modelo_hdp, dictionary=diccionario)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")

Las palabra que definen los tópicos en LDA

0 - sitio, productos, turismo, méxico, doctor, italia, económicas, asunto, común, permitido, participación, acceso, técnicas, connacionales, sentido, situación, control, doctora, persona, comentar
1 - países, méxico, situación, interés, relación, epidemias, región, ahorita, tema, influenza, tipo, país, importantes, hablando, colaboración, economía, enero, principio, técnicos, términos
2 - unidos, insumos, reserva, llegada, minutos, mecanismos, ebrard, connacionales, china, canciller, años, epidemia, periodo, comunidad, útil, febrero, lista, explicar, nacionales, decía
3 - salud, riesgo, años, pública, ocurre, seguridad, equipos, director, situación, nacional, eventos, visto, doctor, oportuno, circunstancia, país, materiales, protección, secretaría, federal
4 - proceso, pendiente, secretaría, ve, secretario, masiva, concentración, sábado, eventos, suspensión, hacienda, fase, completa, medidas, anunciar, agradecimiento, posibles, restricción, e

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
